In [8]:
# =========================================================
# 1. Imports
# =========================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt

In [9]:
# =========================================================
# 2. Load dataset
# =========================================================
df = pd.read_csv("data/online_shoppers_intention.csv")
print("Dataset loaded:", df.shape)

Dataset loaded: (12330, 18)


In [10]:
# =========================================================
# 3. Convert boolean variables to numeric
# =========================================================
df['Revenue'] = df['Revenue'].astype(int)
df['Weekend'] = df['Weekend'].astype(int)


# =========================================================
# 4. Group rare categories in numeric-categorical features
# =========================================================
numeric_categorical = ['OperatingSystems', 'Browser', 'Region', 'TrafficType']

def group_rare(series, threshold=50):
    freq = series.value_counts()
    return series.apply(lambda x: x if freq[x] > threshold else "Other")

for col in numeric_categorical:
    df[col] = df[col].astype(str)
    df[col] = group_rare(df[col])

# =========================================================
# 4. Feature engineering
# =========================================================

# Extracting high season (e.g., Nov-Dec) from Month
df['Is_HighSeason'] = df['Month'].isin(['Nov', 'Dec'])

# =========================================================
# 5. Define categorical & numerical sets
# =========================================================
categorical_nominal = ['Month', 'VisitorType'] + numeric_categorical
numerical_features_original = [
    'Administrative', 'Administrative_Duration',
    'Informational', 'Informational_Duration',
    'ProductRelated', 'ProductRelated_Duration',
    'BounceRates', 'ExitRates', 'PageValues',
    'SpecialDay'
]


# =========================================================
# 6. Train-test split (BEFORE encoding!)
# =========================================================
X = df.drop('Revenue', axis=1)
y = df['Revenue']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, " Test:", X_test.shape)


# =========================================================
# 7. Log-transform duration columns
# =========================================================
for col in ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']:
    X_train[col] = np.log1p(X_train[col])
    X_test[col] = np.log1p(X_test[col])


# =========================================================
# 8. Fit OneHotEncoder on TRAIN split
# =========================================================
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoder.fit(X_train[categorical_nominal])

# Transform
X_train_cat = encoder.transform(X_train[categorical_nominal])
X_test_cat  = encoder.transform(X_test[categorical_nominal])

X_train = X_train.drop(columns=categorical_nominal)
X_test = X_test.drop(columns=categorical_nominal)

# Get feature names (optional)
encoded_cols = encoder.get_feature_names_out(categorical_nominal)


# =========================================================
# Define numerical columns AUTOMATICALLY
# =========================================================
numerical_features = X_train.select_dtypes(include=[np.number]).columns

X_train_num = X_train[numerical_features].reset_index(drop=True)
X_test_num  = X_test[numerical_features].reset_index(drop=True)


X_train_full = np.hstack([X_train_num.values, X_train_cat])
X_test_full  = np.hstack([X_test_num.values,  X_test_cat])



# =========================================================
# 10. Scale numerical + encoded features
# =========================================================
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled  = scaler.transform(X_test_full)


# =========================================================
# 11. PCA (for visualization only)
# =========================================================
pca = PCA(n_components=2)
pca.fit(X_train_scaled)

X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# =========================================================
# 12. SMOTE (APPLY ONLY ON SCALED DATA, ONLY FOR TRAINING MODELS THAT NEED IT)
# =========================================================
# Example: only for tree models, not for LogisticRegression

smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print("After SMOTE:", X_train_smote.shape)


Train: (9864, 18)  Test: (2466, 18)
After SMOTE: (16676, 55)


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_pca.py:606: RuntimeWarning: divide by zero encountered in matmul
  C = X.T @ X
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_pca.py:606: RuntimeWarning: overflow encountered in matmul
  C = X.T @ X
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_pca.py:606: RuntimeWarning: invalid value encountered in matmul
  C = X.T @ X
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encounter

# Model Training

In [11]:
import numpy as np

class RegularizedDiscriminantAnalysis:
    """
    Implementation of Regularized Discriminant Analysis (RDA)
    as described in Friedman (1989).
    
    Parameters
    ----------
    lambda_param : float, default=0.0
        Controls blending between class-specific covariance (0)
        and pooled covariance (1).

    gamma_param : float, default=0.0
        Controls shrinkage of covariance toward spherical matrix.

    Attributes
    ----------
    classes_ : array-like
        Unique class labels.
    priors_ : array-like
        Class prior probabilities.
    cov_ : dict
        Regularized covariance matrix per class.
    means_ : dict
        Class means.
    """

    def __init__(self, lambda_param=0.0, gamma_param=0.0):
        self.lambda_param = lambda_param
        self.gamma_param = gamma_param

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        n_samples, n_features = X.shape

        # Compute class means, covariances
        self.means_ = {}
        covariances = {}
        priors = {}

        for c in self.classes_:
            X_c = X[y == c]
            priors[c] = X_c.shape[0] / n_samples
            self.means_[c] = X_c.mean(axis=0)
            
            # Sample covariance for that class
            covariances[c] = np.cov(X_c, rowvar=False)

        self.priors_ = priors

        # Compute pooled covariance
        pooled_cov = np.zeros((n_features, n_features))
        for c in self.classes_:
            n_c = (y == c).sum()
            pooled_cov += (n_c - 1) * covariances[c]
        pooled_cov /= (n_samples - n_classes)

        # Build regularized covariances (Friedman's RDA)
        self.cov_ = {}

        for c in self.classes_:
            cov_k = covariances[c]

            # Step 1: blend class-specific vs pooled
            cov_lambda = (
                (1 - self.lambda_param) * cov_k
                + self.lambda_param * pooled_cov
            )

            # Step 2: shrink toward spherical covariance
            trace_val = np.trace(cov_lambda) / n_features
            cov_gamma = (
                (1 - self.gamma_param) * cov_lambda
                + self.gamma_param * trace_val * np.eye(n_features)
            )

            self.cov_[c] = cov_gamma

        return self

    def _discriminant(self, X, mean, cov, prior):
        """Compute discriminant function for Gaussian model."""
        inv_cov = np.linalg.inv(cov)
        det_cov = np.linalg.det(cov)
        diff = X - mean

        # Quadratic discriminant function
        term1 = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
        term2 = -0.5 * np.log(det_cov)
        term3 = np.log(prior)

        return term1 + term2 + term3

    def predict(self, X):
        X = np.asarray(X)
        scores = []

        for c in self.classes_:
            s = self._discriminant(
                X,
                self.means_[c],
                self.cov_[c],
                self.priors_[c]
            )
            scores.append(s)

        scores = np.vstack(scores).T
        idx = np.argmax(scores, axis=1)
        return self.classes_[idx]

    def predict_proba(self, X):
        X = np.asarray(X)
        scores = []

        for c in self.classes_:
            s = self._discriminant(
                X,
                self.means_[c],
                self.cov_[c],
                self.priors_[c]
            )
            scores.append(s)

        scores = np.vstack(scores).T
        
        # Convert log-scores to probabilities (softmax)
        exp_scores = np.exp(scores - scores.max(axis=1, keepdims=True))
        return exp_scores / exp_scores.sum(axis=1, keepdims=True)


In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import BernoulliNB
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# ------------------ MODELS ------------------ #
models = {
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000),
    "LDA": LinearDiscriminantAnalysis(priors=[0.5, 0.5]),
    "QDA": QuadraticDiscriminantAnalysis(priors=[0.5, 0.5]),
    "Naive Bayes": BernoulliNB(alpha=10),
    "Perceptron": Perceptron(alpha=1e-6, eta0=0.01, max_iter=1000)
}

# ------------------ RESULTS STORAGE ------------------ #
results = []

for name, model in models.items():

    # Models requiring SMOTE
    if name in ["Naive Bayes", "Perceptron"]:
        clf = Pipeline([
            ("smote", SMOTE(random_state=0)),
            ("clf", model)
        ])
        X_input = X_train  
    else:
        clf = model
        X_input = X_train_scaled

    scores = cross_validate(
        clf,
        X_input,
        y_train,
        cv=10,
        scoring=["f1", "precision", "recall"],
        n_jobs=-1,
        return_train_score=False
    )

    results.append({
        "Model": name,
        "F1 Mean": scores["test_f1"].mean(),
        "F1 Std": scores["test_f1"].std(),
        "Precision Mean": scores["test_precision"].mean(),
        "Precision Std": scores["test_precision"].std(),
        "Recall Mean": scores["test_recall"].mean(),
        "Recall Std": scores["test_recall"].std()
    })

# ------------------ TABLE OUTPUT ------------------ #
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="F1 Mean", ascending=False)

print(df_results.round(3))


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/tania_

                 Model  F1 Mean  F1 Std  Precision Mean  Precision Std  \
1                  LDA    0.627   0.036           0.631          0.034   
0  Logistic Regression    0.613   0.017           0.511          0.014   
3          Naive Bayes    0.579   0.018           0.477          0.019   
4           Perceptron    0.454   0.084           0.342          0.162   
2                  QDA    0.385   0.018           0.247          0.012   

   Recall Mean  Recall Std  
1        0.623       0.042  
0        0.767       0.030  
3        0.736       0.027  
4        0.869       0.129  
2        0.868       0.033  


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/tania_priv/Library

In [19]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    f1_score,
    recall_score,
    classification_report,
    confusion_matrix
)

results = []

for name, model in models.items():
    print(name)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    errors_df = X_test.copy()
    errors_df['y_true'] = y_test.values
    errors_df['y_pred'] = y_pred
    fp = errors_df[(errors_df["y_true"] == 0) & (errors_df["y_pred"] == 1)]
    fn = errors_df[(errors_df["y_true"] == 1) & (errors_df["y_pred"] == 0)]
    print("FP Feature Means:\n", fp[['PageValues', 'ExitRates', 'ProductRelated_Duration']].mean())
    print("FN Feature Means:\n", fn[['PageValues', 'Administrative_Duration', 'BounceRates']].mean())



# --- Create comparison table ---
# results_df = pd.DataFrame(results).sort_values(by="F1_class1", ascending=False)
# print("\nModel comparison on test set:")
# print(results_df.to_string(index=False))

Logistic Regression
FP Feature Means:
 PageValues                 13.019176
ExitRates                   0.019636
ProductRelated_Duration     7.360723
dtype: float64
FN Feature Means:
 PageValues                 3.251775
Administrative_Duration    2.667525
BounceRates                0.012619
dtype: float64
LDA
FP Feature Means:
 PageValues                 24.145849
ExitRates                   0.020953
ProductRelated_Duration     7.620104
dtype: float64
FN Feature Means:
 PageValues                 4.391780
Administrative_Duration    2.729198
BounceRates                0.010051
dtype: float64
QDA
FP Feature Means:
 PageValues                 3.651499
ExitRates                  0.021829
ProductRelated_Duration    6.682301
dtype: float64
FN Feature Means:
 PageValues                 10.932940
Administrative_Duration     2.999583
BounceRates                 0.015437
dtype: float64
Naive Bayes
FP Feature Means:
 PageValues                 13.080756
ExitRates                   0.018312
Produc

/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWarning: divide by zero encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWarning: overflow encountered in matmul
  grad[:n_features] = X.T @ 